In [1]:
import cv2
import numpy as np
import helpers


In [2]:
image = cv2.imread('puzz1.jpg',0)
outerBox = np.ndarray(image.shape)

<a href='https://www.tutorialkart.com/opencv/python/opencv-python-gaussian-image-smoothing/'>Gaussian Blur</a>

In [3]:
image = cv2.GaussianBlur(image,(11,11),0)

<a href="https://pyimagesearch.com/2021/05/12/adaptive-thresholding-with-opencv-cv2-adaptivethreshold/">Adaptive Thresholding</a>

In [4]:
outerBox = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 5, 2)

In [8]:
cv2.imshow('before bitwise',outerBox)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
outerBox = cv2.bitwise_not(outerBox)

<a href='https://docs.opencv.org/3.4/db/df6/tutorial_erosion_dilatation.html'>Dilation</a>

In [15]:
outerBox = cv2.dilate(outerBox, cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3)),iterations=4)

In [16]:
cv2.imshow('here', outerBox)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [28]:
outerBox.shape

(1167, 1125)

In [46]:
contours, heirarchy = cv2.findContours(outerBox, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [47]:
canvas = np.ndarray(image.shape)

canvas = cv2.drawContours(canvas,contours,0,100,4)

In [63]:
contours[0][:,0,:]

array([[  14,   19],
       [  10,   23],
       [  10, 1119],
       [  14, 1123],
       [1110, 1123],
       [1114, 1119],
       [1114,   25],
       [1115,   24],
       [1110,   19]], dtype=int32)

In [64]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4,random_state=0).fit(contours[0][:,0,:])

In [72]:
corners = kmeans.cluster_centers_
corners[:,:]

array([[1112.        , 1121.        ],
       [1113.        ,   22.66666667],
       [  12.        , 1121.        ],
       [  12.        ,   21.        ]])

In [87]:
corners.shape
cornerDict = {'topLeft':(-1,-1), 'topRight':(-1,-1), 'bottomLeft':(-1,-1), 'bottomRight':(-1,-1)}
XmaxMag = [np.linalg.norm(i) for i in corners[:,0]]
YmaxMag = [np.linalg.norm(i) for i in corners[:,1]]
mag = [ (x,y) for x,y in zip(XmaxMag,YmaxMag)]
mag

[(1112.0, 1121.0), (1113.0, 22.66666666666663), (12.0, 1121.0), (12.0, 21.0)]

In [90]:
cornerDict['bottomRight'] = corners[np.argmax([np.linalg.norm(i) for i in mag])]
cornerDict['topLeft'] = corners[np.argmin([np.linalg.norm(i) for i in mag])]
np.argmax(mag,axis=1)

array([1, 0, 1, 1])

In [136]:
def showImage(img):

    #define the screen resulation
    screen_res = 1280, 720
    scale_width = screen_res[0] / img.shape[1]
    scale_height = screen_res[1] / img.shape[0]
    scale = min(scale_width, scale_height)

    #resized window width and height
    window_width = int(img.shape[1] * scale)
    window_height = int(img.shape[0] * scale)

    #cv2.WINDOW_NORMAL makes the output window resizealbe
    cv2.namedWindow('Resized Window', cv2.WINDOW_NORMAL)

    #resize the window according to the screen resolution
    cv2.resizeWindow('Resized Window', window_width, window_height)

    cv2.imshow('Resized Window', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [119]:
tuple(corners[0])

(1112.0, 1121.0)

In [122]:
magarr = np.array(mag)
colors = [(255,0,0), (0,255,0), (0,0,255), (255,255,255)]

copy = image.copy()

In [139]:

point = corners[3]
point = [int(i) for i in point]
copy =  cv2.circle(copy,point,5,(0,255,0),5)
showImage(copy)